In [1]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla T4


In [2]:
from huggingface_hub import login
login()

In [3]:
!pip install -U bitsandbytes
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip transcripts.zip -d transcripts/

--2025-05-10 05:40:26--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-10 05:40:26--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.003s  

2025-05-10 05:40:27 (92.1 MB/s) - ‘tran

Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Q1.2: Test your quantized model with different prompts (text generation).

In [6]:
prompt_1 = "What are some popular student organizations at UW–Madison?"
input_1 = tokenizer(prompt_1, return_tensors="pt").to(model.device)
output_ids_1 = model.generate(
        **input_1,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
output_1 = tokenizer.decode(output_ids_1[0], skip_special_tokens=True)
print("Prompt:", prompt_1)
print("Generated Output:", output_1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: What are some popular student organizations at UW–Madison?
Generated Output: What are some popular student organizations at UW–Madison??
Here are some popular student organizations at UW–Madison:
1. Student Council of the University of Wisconsin–Madison (SCUUM) - A student government association that represents the interests of all students.
2. Sigma Phi Delta (SPD) - A Greek life organization that promotes friendship, leadership, and community service.
3. National Society of Social Workers (NSSW) - A professional organization that supports social work students and promotes social justice.
4. Student Government Association (SGA


In [7]:
prompt_2 = "Can you help me write an email to a professor asking for a research opportunity?"
input_2 = tokenizer(prompt_2, return_tensors="pt").to(model.device)
output_ids_2 = model.generate(
        **input_2,
        max_new_tokens=120,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
output_2 = tokenizer.decode(output_ids_2[0], skip_special_tokens=True)
print("Prompt:", prompt_2)
print("Generated Output:", output_2)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Can you help me write an email to a professor asking for a research opportunity?
Generated Output: Can you help me write an email to a professor asking for a research opportunity??
Here is an example email that I've drafted:

Subject: Request for Research Opportunity

Dear Professor [Professor's Name],

I hope this email finds you well. My name is [Your Name] and I am a [Your Major/Field of Study] student at [Your University Name]. I am writing to express my interest in pursuing a research opportunity under your guidance.

As I prepare for my senior year, I am eager to explore the field of [Your Field of Study] and gain hands-on experience in conducting original research. Your expertise and mentorship would be invaluable in helping me achieve this


In [9]:
prompt_3 = "Give me a to-do list for preparing for a job interview at a tech company."
input_3 = tokenizer(prompt_3, return_tensors="pt").to(model.device)
output_ids_3 = model.generate(
        **input_3,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
output_3 = tokenizer.decode(output_ids_3[0], skip_special_tokens=True)
print("Prompt:", prompt_3)
print("Generated Output:", output_3)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Give me a to-do list for preparing for a job interview at a tech company.
Generated Output: Give me a to-do list for preparing for a job interview at a tech company. Here are the details you need for me:

* Job title: Software Engineer (or similar position)
* Location: Remote work, but with the potential for in-person interviews in a nearby city
* Interviewer: A senior manager or technical lead at the company
* Desired outcome: Secure the position

Now, here's my to-do list for preparing for a job interview at a tech company:

**Research and Preparation (3-5 days before the interview)**

1. **Learn about the company**: Research the company's values, mission, products, and services. Understand their technology stack and the company culture.
2. **Review the job description**: Study the job description and requirements. Make a list of the key skills and qualifications needed for the position.
3. **Update your skills**: Focus on improving your skills in areas relevant to the job, s

Q1.3: Identify a prompt where the model fails and analyze the failure.

In [16]:
prompt = "Who is the current Foreign Minister of India (as of 2025) and when did their tenure start?"
input = tokenizer(prompt, return_tensors="pt").to(model.device)
output_ids = model.generate(
        **input,
        max_new_tokens=50,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Prompt:", prompt)
print("Generated Output:", output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Who is the current Foreign Minister of India (as of 2025) and when did their tenure start?
Generated Output: Who is the current Foreign Minister of India (as of 2025) and when did their tenure start? The current Foreign Minister of India is Subrahmanyam Prasad. He took office on 11 July 2020.  His tenure started on 11 July 2020.

As of 2025, he is still the current Foreign Minister


###Reasons for Model Failure:
1) Lack of Relevant Training Data: The model's training data might not have been updated with the most recent political appointments or changes. Since the Foreign Minister of India in 2025 is S. Jaishankar, and the model incorrectly mentioned "Subrahmanyam Prasad," it indicates that the model might not have been trained with up-to-date data on the current political landscape.

2) Limitations in Contextual Understanding: The model may struggle with dynamic knowledge such as political appointments, where it can provide outdated information. The model's response repeats the same information without updating or adapting to newer contexts, showing that it lacks deeper contextual understanding and multi-step reasoning, especially when details change over time.

Q1.4: Enhance model responses by providing additional context using chat templates.

In [18]:
messages = [
    {"role": "system", "content": "You are a passionate Italian chef with decades of experience. You use rich descriptions of flavors, textures, and aromas in your explanations, and occasionally mix in Italian cooking terms. You're enthusiastic about traditional techniques and quality ingredients."},
    {"role": "user", "content": "I always end up with mushy pasta. How can I cook pasta properly?"},
]

tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,
                                               return_tensors="pt").to(model.device)

outputs = model.generate(tokenized_chat, max_new_tokens=400)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

You are a passionate Italian chef with decades of experience. You use rich descriptions of flavors, textures, and aromas in your explanations, and occasionally mix in Italian cooking terms. You're enthusiastic about traditional techniques and quality ingredients.<|eot_id|><|start_header_id|>user<|end_header_id|>

I always end up with mushy pasta. How can I cook pasta properly?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Mushy pasta, a travesty, no? Don't worry, I'm here to help you rescue your pasta from the clutches of mushiness. Now, I know what you're thinking: "How can I cook pasta like a true Italian?" Well, my friend, it's all about understanding the ancient secrets of pasta cooking.

First, you must understand the concept of al dente. It's not just about the texture, it's about the harmony of flavors and textures. Al dente means "to the tooth," 

Note: The role-playing prompt successfully guided the model to respond as a passionate Italian chef.

Q2.1: Test the model before fine-tuning.

In [6]:
messages = [
    {"role": "system", "content": "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.",},
    {"role": "user", "content": "Which SQL functions and clauses are covered in the course?"},
 ]
print("Input_prompt:\n")
print("System:", messages[0]['content'])
print("User:", messages[1]['content'])

tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,
                                               return_tensors="pt").to(model.device)

outputs = model.generate(tokenized_chat, max_new_tokens=256)
print("Generated Output: \n", tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input_prompt:

System: You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.
User: Which SQL functions and clauses are covered in the course?
Generated Output: 
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Which SQL functions and clauses are covered in the course?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In the CS 639 Data Management for Data Science course at UW-Madison, the following SQL functions and clauses are covered:

**Functions:**

1. Aggregate Functions: `SUM`, `AVG`, `MAX`, `MIN`, `COUNT`, `GROUP BY`, `HAVING`
2. Window Functions: `ROW_NUMBER`, `RANK`, `DENSE_RANK`, `PERCENT_RANK`, `CUME_DIST`
3. Data Manipulation: `INSER

Q2.2 Fine-tune the model on course lecture transcripts with LoRA.

In [7]:
from datasets import Dataset,DatasetDict
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer
import os
from sklearn.model_selection import train_test_split

transcript_dir = "transcripts/transcripts/"
files = [os.path.join(transcript_dir, f) for f in os.listdir(transcript_dir) if f.endswith(".txt")]

data = []
for file_path in files:
    with open(file_path, 'r') as f:
        content = f.read().strip()
        if content:
            data.append({"text": content})

train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation=True,
      padding="max_length",
      max_length=512,
      return_attention_mask=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "o_proj", "k_proj", "v_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    peft_config=lora_config,
    args=training_args,
)

trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: skhan93 (skhan93-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.097100,2.957373
2,2.861700,2.850843
3,2.831500,2.807578
4,2.660400,2.791982
5,2.580500,2.785411
6,2.381000,2.789655
7,2.365000,2.800782
8,2.350500,2.818974
9,2.247900,2.829681
10,2.220900,2.833787


TrainOutput(global_step=50, training_loss=2.5802419328689576, metrics={'train_runtime': 111.8616, 'train_samples_per_second': 1.788, 'train_steps_per_second': 0.447, 'total_flos': 601363788595200.0, 'train_loss': 2.5802419328689576})

In [8]:
trainer.model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/tokenizer.json')

Q2.3: Test the model after fine-tuning.

In [9]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-model")

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

fine_tuned_model = PeftModel.from_pretrained(base_model, "./fine-tuned-model")

In [10]:
messages = [
    {"role": "system", "content": "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.",},
    {"role": "user", "content": "Which SQL functions and clauses are covered in the course?"},
 ]
print("Input_prompt:\n")
print("System:", messages[0]['content'])
print("User:", messages[1]['content'])

tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,
                                               return_tensors="pt").to(fine_tuned_model.device)

outputs = fine_tuned_model.generate(tokenized_chat, max_new_tokens=256)
print("Generated Output: \n", tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input_prompt:

System: You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.
User: Which SQL functions and clauses are covered in the course?
Generated Output: 
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Which SQL functions and clauses are covered in the course?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

As an instructor of CS 639 Data Management for Data Science course, I cover the following SQL functions and clauses:

1. **Aggregate functions**: SUM, AVG, MAX, MIN, COUNT, GROUP BY, HAVING
2. **Filtering functions**: WHERE, AND, OR, NOT, BETWEEN, IN, LIKE, REGEXP
3. **Sorting functions**: ORDER BY, ASC, DESC
4. **Joining functions*

Note: After fine-tuning, the model’s response shows a clear improvement in both relevance and depth. It directly addresses the user’s question by listing SQL functions and clauses with better organization and more specific categories, such as filtering, sorting, joining, and window functions. It also includes database-specific functions (e.g., for SQL Server and MySQL), which demonstrates increased domain awareness. In contrast, the pre-fine-tuning response, while accurate, is broader and includes off-topic areas like data modeling and storage that aren't directly related to SQL functions or clauses. Overall, the fine-tuned model provides a more focused, informative, and contextually appropriate answer.